In [1]:
import pandas as pd
import numpy as np

In [2]:
naf = pd.read_csv("NAF_Matches_2013-14.csv")

In [4]:
naf.head()

,tournamentname,tournamentnation,tournamentcity,gameid,HomeCoachNAFname,HomeCoachNAFid,HomeRace,goalshome,AwayCoachNAFname,AwayCoachNAFid,AwayRace,goalsaway,gamedate
0,Bash-N-Dash Classic,United States,Cary,99942,Sherrirealtor,19508,Wood Elves,0,d4rkt3mpl4r,19509,Chaos Dwarves,3,05/01/2013
1,Bash-N-Dash Classic,United States,Cary,99943,Sugarwookie,2444,Chaos Pact,1,Gaixo,13696,Dark Elves,3,05/01/2013
2,Bash-N-Dash Classic,United States,Cary,99944,jccaran,19506,Orc,1,orgavin,19083,Khemri,0,05/01/2013
3,Bash-N-Dash Classic,United States,Cary,99945,interociter_1955,19507,Khemri,2,profgoldfinch,18217,Ogres,1,05/01/2013
4,Bash-N-Dash Classic,United States,Cary,99946,ericbehymer,19164,Orc,2,xxthelizardkingxx,18830,Lizardmen,1,05/01/2013


In [5]:
# find r1 games, defined as the first game played in a given tournament by the home coach
grps = naf.groupby(by="tournamentname")

In [52]:
race_names = sorted(set(naf.HomeRace.tolist()))

In [53]:
race_ids = {race:idf for race, idf in zip(race_names, range(len(race_names)))}

In [33]:
def compute_wr_first():

    result = np.zeros((24, 24), dtype="float")
    count = np.zeros((24, 24), dtype="int")

    # or take the numbe of coaches and divide by 2. Take the first x games.
    for g in grps.groups:
        x = grps.get_group(g)
        set_coaches = set(x.HomeCoachNAFid.tolist() + x.AwayCoachNAFid.tolist())
        ngames = len(set_coaches)//2

        for i, row in x[:ngames].iterrows():
            id_home = race_ids[row.HomeRace]
            id_away = race_ids[row.AwayRace]
            count[id_home, id_away] = count[id_home, id_away] + 1
            count[id_away, id_home] = count[id_away, id_home] + 1

            if row.goalshome > row.goalsaway:
                result[id_home, id_away] = result[id_home, id_away] + 1
            elif row.goalshome < row.goalsaway:
                result[id_away, id_home] = result[id_away, id_home] + 1
            else:
                result[id_home, id_away] = result[id_home, id_away] + 0.5
                result[id_away, id_home] = result[id_away, id_home] + 0.5
                
    return result, count, result/count

In [34]:
def compute_wr_last():

    result = np.zeros((24, 24), dtype="float")
    count = np.zeros((24, 24), dtype="int")

    # or take the numbe of coaches and divide by 2. Take the first x games.
    for g in grps.groups:
        x = grps.get_group(g)
        set_coaches = set(x.HomeCoachNAFid.tolist() + x.AwayCoachNAFid.tolist())
        ngames = len(set_coaches)//2

        for i, row in x[-ngames:].iterrows():
            id_home = race_ids[row.HomeRace]
            id_away = race_ids[row.AwayRace]
            count[id_home, id_away] = count[id_home, id_away] + 1
            count[id_away, id_home] = count[id_away, id_home] + 1

            if row.goalshome > row.goalsaway:
                result[id_home, id_away] = result[id_home, id_away] + 1
            elif row.goalshome < row.goalsaway:
                result[id_away, id_home] = result[id_away, id_home] + 1
            else:
                result[id_home, id_away] = result[id_home, id_away] + 0.5
                result[id_away, id_home] = result[id_away, id_home] + 0.5
                
    return result, count, result/count

In [41]:
_, fc, first = compute_wr_first()
_, lc, last = compute_wr_last()

In [46]:
x = np.where(counts > 50)

In [55]:
race_ids

{'Amazons': 0,
 'Chaos': 1,
 'Chaos Dwarves': 2,
 'Chaos Pact': 3,
 'Dark Elves': 4,
 'Dwarves': 5,
 'Elves': 6,
 'Goblins': 7,
 'Halflings': 8,
 'High Elves': 9,
 'Humans': 10,
 'Khemri': 11,
 'Lizardmen': 12,
 'Necromantic': 13,
 'Norse': 14,
 "Nurgle's Rotters": 15,
 'Ogres': 16,
 'Orc': 17,
 'Skaven': 18,
 'Slann': 19,
 'Undead': 20,
 'Underworld': 21,
 'Vampires': 22,
 'Wood Elves': 23}

In [59]:
counts = fc + lc

for (i, j) in zip(*np.where(counts > 40)):
    if np.in1d((i, j), (7, 8, 16)).any():
        if i > j:
            print(race_names[i], race_names[j], first[i, j], last[i, j], fc[i, j], lc[i, j])

Goblins Chaos Dwarves 0.444444444444 0.395833333333 18 24
Goblins Dark Elves 0.211538461538 0.260869565217 26 23
Goblins Dwarves 0.120689655172 0.386363636364 29 22
Halflings Goblins 0.53125 0.51724137931 16 29
Humans Goblins 0.704545454545 0.666666666667 22 21
Lizardmen Goblins 0.711538461538 0.7 26 20
Norse Goblins 0.64 0.5 25 34
Norse Halflings 0.56 0.5625 25 16
Ogres Dwarves 0.285714285714 0.285714285714 21 21
Ogres Lizardmen 0.326086956522 0.333333333333 23 24
Ogres Norse 0.282608695652 0.282608695652 23 23
Orc Goblins 0.603448275862 0.680555555556 29 36
Orc Halflings 0.5 0.630434782609 20 23
Orc Ogres 0.638888888889 0.586206896552 18 29
Skaven Goblins 0.833333333333 0.72 24 25
Undead Ogres 0.759259259259 0.729166666667 27 24
Wood Elves Goblins 0.714285714286 0.90625 28 16
